# LLM VectorDB Test #1
#### w/ FAISS

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lgdx_team2")

LangSmith 추적을 시작합니다.
[프로젝트명]
lgdx_team2


In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd

# 랭체인 환경 설정
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# VectorDB - FAISS
from langchain_community.vectorstores import FAISS


In [4]:
### 01. CSV 파일에서 문서 로드 ###
loader = CSVLoader('../../data/movie_4000_preprocessed.csv', encoding='utf8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

### 02. pandas로 데이터프레임 칼럼명 가져오기
csv_path = '../../data/movie_4000_preprocessed.csv'
df = pd.read_csv(csv_path, encoding='utf8')
colnames = df.columns

문서의 수: 3901


In [5]:
### 03. 메타데이터 추가 ###
docs = []
for _, row in df.iterrows():
  # 필요한 메타데이터 설정
  metadata = {
    'title': row['movie_title'],
    'genre': row['genre']
  }
  # 각 행의 데이터를 문서로 변환
  doc = Document(
    page_content=str(row.to_dict()),
    metadata=metadata
  )
  docs.append(doc)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[100].metadata)

문서의 수: 3901
[메타데이터 예시]
 {'title': '아름답다', 'genre': '드라마'}


In [8]:
### 04. 데이터 청크 나누기 ###
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1090, chunk_overlap=0
)
splits = text_splitter.split_documents(docs)
print("split된 문서의 수:", len(splits))

split된 문서의 수: 3901


In [4]:
### 05. 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\USER\AppData\Local\Temp\ipykernel_5820\3636801640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')
c:\workspaces\LGDXteam2\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
### 06. 벡터스토어 생성 ###
## FAISS
vectorstore = FAISS.from_documents(
  documents=splits,
  embedding=embeddings,
)
print("FIASS 저장 완료!")

FIASS 저장 완료!


In [12]:
# 벡터스토어 로컬에 저장
vectorstore.save_local("../movie_4000_vectorstore_faiss")

In [5]:
# 벡터스토어 로드
new_vector_store = FAISS.load_local("../movie_4000_vectorstore_faiss",
                                    embeddings=embeddings,
                                    allow_dangerous_deserialization=True)

In [18]:
# 검색 테스트
# FAISS는 메타데이터 필터링을 지원하지 않음!!
results = new_vector_store.similarity_search_with_score(
    query="연쇄살인범을 쫓는 형사",
    k=5
)

# 유사도 값 높은 순으로 정렬 (score 기준 내림차순)
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

# 결과 출력
for doc, score in sorted_results:
    print(f"🎬 제목: {doc.metadata.get('title', 'N/A')}")
    print(f"🎭 장르: {doc.metadata.get('genre', 'N/A')}")
    # print(f"📄 요약: {doc.page_content[:200]}...")  # 요약 일부만 출력
    print(f"⭐ 유사도 점수: {score:.4f}")  # 소수점 4자리까지 출력
    print("=" * 40)  # 구분선

🎬 제목: 테이킹 라이브즈
🎭 장르: 공포/스릴러
⭐ 유사도 점수: 0.5304
🎬 제목: 살인의뢰
🎭 장르: 공포/스릴러
⭐ 유사도 점수: 0.5189
🎬 제목: 폭력형경
🎭 장르: 액션/어드벤쳐
⭐ 유사도 점수: 0.5139
🎬 제목: 살인자의 기억법
🎭 장르: 공포/스릴러
⭐ 유사도 점수: 0.5071
🎬 제목: 실종
🎭 장르: 공포/스릴러
⭐ 유사도 점수: 0.4910


<br><hr>

In [19]:
### 03. 랭체인 구성 ###
# 검색기 생성
retriever = new_vector_store.as_retriever(
    search_type="mmr",   
    search_kwargs={"k": 20,              # 반환할 문서 수 (default: 4)
                   "fetch_k": 50,       # MMR 알고리즘에 전달할 문서 수
                   "lambda_mult": 0.5,    # 결과 다양성 조절 (default: 0.5),
                   }
)

# 프롬프트 템플릿 설정
template = """
You are a movie-recommendation chatbot.
You must only answer based on the given context.
Do not generate answers that are not directly supported by the context.

[Context]:
{retrieved_context}

[Question]:
{query}

[Answer]:
"""
prompt = ChatPromptTemplate.from_template(template)

# Google Gemini 모델 생성
def load_gemini(system_instruction):
    model = ChatGoogleGenerativeAI(
        model='gemini-1.5-flash',
        temperature=0.3,
        max_tokens=5000,
        system_instruction=system_instruction,
        api_key=GEMINI_API_KEY
    )
    print(">>>>>>> model loaded...")
    return model

system_instruction = """you are a movie-recommendation chatbot. you must answer based on given data."""
llm = load_gemini(system_instruction)

>>>>>>> model loaded...


In [20]:
# langchain 체인 구성
rag_chain = (
  {"query":RunnablePassthrough(),
    "retrieved_context": retriever,
  }
  # question(사용자의 질문) 기반으로 연관성이 높은 문서 retriever 수행 >> format_docs로 문서를 하나로 만듦
  | prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | llm                  # llm이 원하는 답변을 만듦
  | StrOutputParser()
)

In [26]:
### 04. 모델 테스트 ###
user_query='외계인이 나오는 영화를 추천해줘'
response = rag_chain.invoke(user_query)
print(response)

`캡티브 스테이트`와 `에일리언 2020`을 추천합니다.

